In [2]:
# !pip install opencv-python
# !pip install numpy
# !pip install mediapipe opencv-python
# !pip install pandas
# !pip install xmltodict
# !pip install lxml

In [6]:
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#mediapipe
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Yolo 로드
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

## XML 정보 불러오기

In [7]:
from lxml import etree
from PIL import Image
import xmltodict, cv2, os
import xml.etree.ElementTree as ET

In [68]:
#xml 파일 불러옴
def load_xmldata(video_name):
    xml_file = '{0}.xml'.format(video_name)
    doc = ET.parse(xml_file)
    root = doc.getroot()
    frame_list = []
    action_list = []
    action_name = ['normal']

    action_list=root.findall("./object/action")
    for i in action_list:
        for frame in i.findall("frame"):
            frame_list.append([i.find("actionname").text, int(frame.find("start").text), int(frame.find("end").text)])
    for action in root.iter("action"):
        action_name.append(action.find("actionname").text)
        
    make_dir(video_name, action_name)
    return frame_list

In [70]:
def frame_cutting(video_dir, frame_list):
    vidcap = cv2.VideoCapture(video_dir)
    root_path = ntpath.basename(video_dir)
    
    count = 1
    success = True

    while success:
        try:
            success, image = vidcap.read()
            if(int(vidcap.get(1)) % 1 == 0):
                #print('Saved frame number : ' + str(int(vidcap.get(1))))
                count += 1
                for i in frame_list:
                    if i[1]<= count and count <= i[2]:
                        cv2.imwrite("{0}/{1}/{2}.jpg".format(root_path, i[0], count), image)
#                         print('Saved frame%d.jpg' % count)
                    else: continue
        except: pass
    vidcap.release()

## 폴더 생성

In [50]:
#name 폴더 만들기
def make_dir(root_dir, dir_name):
    folder_list = []
    for i in dir_name:
        try: 
            if i not in folder_list:
#                 print("./{0}/{1}".format(root_dir, dir_name))
                os.makedirs("./{0}/{1}".format(root_dir, dir_name))
                folder_list.append(dir_name)

        except:
            pass
    return folder_list

## 파일 검색

In [59]:
import pathlib
import os
import ntpath

def find_file_fullname(fileDir):
    file_name = []
    for name in os.listdir(fileDir):
        file_name.append(fileDir + name)
    return file_name

def find_mp4(fileDir, fileExt):
    video_name = []
    for name in pathlib.Path(fileDir).glob(fileExt):
        video_name.append(str(name))
    return video_name

def path_to_filename(fileDir):
    file_name = []
    for name in fileDir:
        file_name.append(ntpath.basename(name))
    return file_name

## csv 쓰기

In [ ]:
#미완

def w_csv(video_name, row):
    f = open('{0}_landmark.csv'.format(video_name),'w', newline='')
    wr = csv.writer(f)
    wr.writerow(row)
    f.close()

#     f = open('landmark.csv','a', newline='')
#     wr = csv.writer(f)
#     wr.writerow([3,'최번개', '강원'])
#     f.close()

In [ ]:
img = cv2.imread("C:/Users/BVer/are_you_ok/img/pose10.jpg")
#img = cv2.resize(img, None, fx=0.4, fy=0.4)
height, width, channels = img.shape
def pose_detect(img):
    img.flags.writeable = True
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    blob = cv2.dnn.blobFromImage(img, 0.00392, (512, 512), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []
    for out in outs: #yolo object detect
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # 좌표
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    new_img = img
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            if(label == 'person'):
                #frame = img[y:y+h, x:x+w]
                frame = img[y-3:y+h+6, x-3:x+w+6]
                
                #mediapipe pose detect on yolo object frame
                with mp_pose.Pose(model_complexity=2, min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
                    # Recolor image to RGB
                    smallimg = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    smallimg.flags.writeable = False

                    # Make detection
                    results = pose.process(smallimg)

                    # Render detections
                    #mp_drawing.draw_landmarks(smallimg, results.pose_landmarks, mp_pose.POSE_CONNECTIONS) #조각별로 보기
                    paint_landmark(results.pose_landmarks, x, y, w, h, width, height)
                    mp_drawing.draw_landmarks(new_img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
#                     plt.imshow(smallimg)
#                     plt.show()
#     plt.imshow(new_img)
#     plt.show()

        #Recolor back to BGR plt로 출력할땐 주석처리
    new_img.flags.writeable = True
    new_img = cv2.cvtColor(new_img, cv2.COLOR_RGB2BGR)
    return new_img


    new_filename = '112'
    cv2.imwrite('new_img/{0}.jpg'.format(new_filename), new_img)

In [ ]:
def paint_landmark(landmarks, x, y, w, h, ori_w, ori_h):
    marks = []
    for lndmrk in mp_pose.PoseLandmark:
        try:
            if landmarks.landmark[lndmrk.value].visibility >= 0.5: #min_detection_confidence
                landmarks.landmark[lndmrk.value].x = ((x - 3) + (w + 6) * landmarks.landmark[lndmrk.value].x)/ori_w
                
                landmarks.landmark[lndmrk.value].y = ((y - 3) + (h + 6) * landmarks.landmark[lndmrk.value].y)/ori_h
                
        except:
            pass

# 파일별 폴더 생성

In [ ]:
try: #name 폴더 만들기
    os.makedirs("./preprocessing")
except:
    pass

root_dir = 'C:/Users/BVer/are_you_ok'
video_dir = root_dir + '/video/assult/insidedoor_01/'

sub_folder_name = []
sub_folder_name = find_file_fullname(video_dir)

for name in sub_folder_name:
    video_path = []
    video_path = find_mp4(name, r'*.mp4')
#     print(video_path)
    make_dir(name, path_to_filename(video_path))

    for file in video_path:
        #xml 파일 정보 확인
        frame_cutting(file, load_xmldata(file[:-4]))
    
#     #프레임 이미지에서 스켈레톤 따기 (프레임 수 확인해서 엑션에 있으면 각각의 엑션 폴더에 아니면 일반 폴더에 이미지 저장, csv에 행동명과 landmark 좌표 저장)

./C:/Users/BVer/are_you_ok/video/assult/insidedoor_01/10-1/['10-1_cam01_assault03_place07_night_spring.mp4', '10-1_cam01_assault03_place07_night_summer.mp4', '10-1_cam01_assault03_place07_night_winter.mp4', '10-1_cam02_assault03_place07_night_spring.mp4', '10-1_cam02_assault03_place07_night_summer.mp4', '10-1_cam02_assault03_place07_night_winter.mp4', '10-1_cam03_assault03_place07_night_spring.mp4', '10-1_cam03_assault03_place07_night_summer.mp4', '10-1_cam03_assault03_place07_night_winter.mp4']
./C:/Users/BVer/are_you_ok/video/assult/insidedoor_01/10-1/['10-1_cam01_assault03_place07_night_spring.mp4', '10-1_cam01_assault03_place07_night_summer.mp4', '10-1_cam01_assault03_place07_night_winter.mp4', '10-1_cam02_assault03_place07_night_spring.mp4', '10-1_cam02_assault03_place07_night_summer.mp4', '10-1_cam02_assault03_place07_night_winter.mp4', '10-1_cam03_assault03_place07_night_spring.mp4', '10-1_cam03_assault03_place07_night_summer.mp4', '10-1_cam03_assault03_place07_night_winter.mp4'